In [2]:
import tensorflow as tf
import numpy as np
import os,glob,cv2
import sys,argparse
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

image_size=128
num_channels=3

match_bad=0
match_good=0

bad_scores0=[]
bad_scores1=[]


/home/lishuai/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/lishuai/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
for img_path in glob.glob("./test/bad/*.jpg"):
    images=[]
    
    image=cv2.imread(img_path)
    image=cv2.resize(image,(image_size,image_size),0,0,cv2.INTER_LINEAR)
    images.append(image)
    
    images=np.array(images,dtype=np.uint8)
    images=images.astype('float32')
    images=np.multiply(images,1.0/255.0)
    
    x_batch=images.reshape(1,image_size,image_size,num_channels)
    
    sess=tf.Session()

    saver=tf.train.import_meta_graph('./good-bad-model.meta')
    saver.restore(sess,tf.train.latest_checkpoint('./'))
    
    graph=tf.get_default_graph()
    
    y_pred=graph.get_tensor_by_name('y_pred:0')
    x=graph.get_tensor_by_name('x:0')
    
    y_true=graph.get_tensor_by_name('y_true:0')
    y_test_images=np.zeros((1,2))
    
    feed_dict_testing={x:x_batch,y_true:y_test_images}
    result=sess.run(y_pred,feed_dict=feed_dict_testing)
    
    print('Models output\n',result)
    bad_scores0.append(result[0][0])
    index=result.argmax(axis=1)
    if index[0]==0:
        match_bad+=1
print('match_bad is %d images'%match_bad)

INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[0.00969777 0.99030226]]
INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[0.00214074 0.9978593 ]]
INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[5.9163111e-04 9.9940836e-01]]
INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[0.02333989 0.97666013]]
INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[0.13607939 0.8639206 ]]
INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[0.00367462 0.9963253 ]]
INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[0.00269928 0.9973008 ]]
INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[0.03011246 0.96988755]]
INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[0.00302744 0.99697256]]
INFO:tensorflow:Restoring parameters from ./good-bad-model
Models output
 [[0.01200433 0.98799

KeyboardInterrupt: 

In [3]:
for img_path in glob.glob("./test/good/*.jpg"):
    images=[]
    image=cv2.imread(img_path)
    image=cv2.resize(image,(image_size,image_size),0,0,cv2.INTER_LINEAR)
    images.append(image)

    
    images=np.array(images,dtype=np.uint8)
    images=images.astype('float32')
    images=np.multiply(images,1.0/255.0)
    
    x_batch=images.reshape(1,image_size,image_size,num_channels)
    sess=tf.Session()
    
    #saver=tf.train.import_meta_graph('./weights3/good-bad-model.meta')
    #saver.restore(sess,tf.train.latest_checkpoint('./weights3'))
    
    saver=tf.train.import_meta_graph('./good-bad-model.meta')
    saver.restore(sess,tf.train.latest_checkpoint('./'))
    
    graph=tf.get_default_graph()
    y_pred=graph.get_tensor_by_name('y_pred:0')
    x=graph.get_tensor_by_name('x:0')
    
    y_true=graph.get_tensor_by_name('y_true:0')
    y_test_images=np.zeros((1,2))
    
    feed_dict_testing={x:x_batch,y_true:y_test_images}
    result=sess.run(y_pred,feed_dict=feed_dict_testing)
    
    print(result,result.shape,result[0][0])
    bad_scores1.append(result[0][0])
    print(result.argmax(axis=1))
    print(result.argmax(axis=1))
    index=result.argmax(axis=1)
    if index[0]==1:
        match_good+=1
print('match good is %d images'%match_good)

match good is 0 images


In [4]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
# Generate the appropriate format for evaluation 
y_bad=np.ones([30])
y_good=np.ones([116])
y_bad-=1
print(y_bad,y_bad.shape)
print(y_good,y_good.shape)
y_label=np.concatenate((y_bad,y_good),axis=0)
predict_scores=np.concatenate((bad_scores0,bad_scores1),axis=0)


average_precision = average_precision_score(y_label, predict_scores)
print('Average precision-recall score: {0:0.2f}'.format(average_precision))
precision, recall, _ = precision_recall_curve(y_label, predict_scores)
plt.step(recall, precision, color='b', alpha=0.2,where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
plt.savefig('Precision_Recall')
plt.show()


#Draw the ROC curve and calculate the AUC for evaluating the model
# fpr,tpr,thresholds=metrics.roc_curve(y_label,predict_scores,pos_label=0)
# plt.plot(fpr,tpr)
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.savefig('Results')
# plt.show()

# AUC=roc_auc_score(y_label,predict_scores)
# print(AUC)


[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] (30,)
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.] (116,)


ValueError: Found input variables with inconsistent numbers of samples: [146, 30]